##Get data from Kaggle

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/CornLeafDisease/kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d smaranjitghose/corn-or-maize-leaf-disease-dataset

 88% 142M/161M [00:01<00:00, 108MB/s]
100% 161M/161M [00:01<00:00, 88.8MB/s]


##Placeholder

In [7]:
  src_zip = "corn-or-maize-leaf-disease-dataset.zip"
  DATA_SRC = "data" # (technically a subset of only 10K images)
  IMAGES_PER_LABEL = 500
  BALANCED_SPLITS = {"train" : 400, "val" : 50, "test": 50}


In [8]:
%%capture
!unzip $src_zip

##WandB

In [9]:
!pip install wandb -qU
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 28.2 MB/s 
     |████████████████████████████████| 181 kB 55.8 MB/s 
     |████████████████████████████████| 153 kB 76.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
import os
from random import shuffle
import numpy as np

# source directory for all raw data
SRC = DATA_SRC
PREFIX = "cornset" # convenient for tracking local data
PROJECT_NAME = "corn_leaf_photos"

# number of images per class label
# the total number of images is 4X this (4 classes)
TOTAL_IMAGES = IMAGES_PER_LABEL * 4

In [11]:
labels = os.listdir(SRC)

In [12]:
# if this is a substantially new dataset, give it a new name
# this will create a whole new placeholder (Artifact) for this dataset
# instead of just incrementing a version of the old dataset
RAW_DATA_AT = "_".join([PREFIX, "raw_data", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="upload")
# create an artifact for all the raw data
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")

# SRC_DIR contains 10 folders, one for each of 10 class labels
# each folder contains images of the corresponding class
labels = os.listdir(SRC)
for l in labels:
  imgs_per_label = os.path.join(SRC, l)
  if os.path.isdir(imgs_per_label):
    # filter out "DS_Store"
    imgs = [i for i in os.listdir(imgs_per_label) if not i.startswith(".DS")]
    # randomize the order
    shuffle(imgs)
    img_file_ids = imgs[:IMAGES_PER_LABEL]
    for f in img_file_ids:
      file_path = os.path.join(SRC, l, f)
      # add file to artifact by full path
      raw_data_at.add_file(file_path, name=l + "/" + f)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

wandb: Currently logged in as: jordaocassiano (gabrieljordao). Use `wandb login --relogin` to force relogin


##Split raw data to prepare for training

In [13]:
# if this is a substantially different dataset, give it a new name
# this will create a whole new placeholder (Artifact) for this split
# instead of just incrementing a version of the old data split
SPLIT_DATA_AT = "_".join([PREFIX, "80-10-10", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="data_split")

# create balanced train, val, test splits
# each count is the number of images per label
SPLIT_COUNTS = {"train" : 400, "val" : 50, "test": 50}

# find the most recent ("latest") version of the full raw data
# you can of course pass around programmatic aliases and not string literals
# note: RAW_DATA_AT is defined in the previous cell—if you're running
# just this step, you may need to hardcode it
data_at = run.use_artifact(RAW_DATA_AT + ":latest")
# download it locally (for illustration purposes/across hardware; you can
# also sync/version artifacts by reference)
data_dir = data_at.download()

data_split_at = wandb.Artifact(SPLIT_DATA_AT, type="balanced_data")

# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label", "split"])

labels = os.listdir(data_dir)
for l in labels:
  if l.startswith("."): # skip non-label file
    continue
  imgs_per_label = os.listdir(os.path.join(data_dir, l))
  shuffle(imgs_per_label)
  start_id = 0
  for split, count in SPLIT_COUNTS.items():
    # take a subset
    split_imgs = imgs_per_label[start_id:start_id+count]
    for img_file in split_imgs:
      f_id = img_file.split(".")[0]
      full_path = os.path.join(data_dir, l, img_file)
      # add file to artifact by full path
      # note: pass the label to the name parameter to retain it in
      # the data structure 
      data_split_at.add_file(full_path, name = os.path.join(split, l, img_file))
      # add a preview of the image
      if split != "test":
        preview_dt.add_data(f_id, wandb.Image(full_path), l, split)
      else:
        # pretend we have unlabeled test data
        # (replace "unknown" with l if you'd like to keep the labels :)
        preview_dt.add_data(f_id, wandb.Image(full_path), l, split)
    start_id += count

# log artifact to W&B
data_split_at.add(preview_dt, "data_split")
run.log_artifact(data_split_at)
run.finish()

wandb: Downloading large artifact cornset_raw_data_2000:latest, 70.66MB. 2000 files... Done. 0:0:0.9


## Train Model

In [14]:
# EXPERIMENT CONFIG
#------------------------
# Core globals to modify
NUM_EPOCHS = 5 


# optional globals to modify
# set to a custom name to help keep your experiments organized
RUN_NAME = "" 
# change this if you'd like start a new set of comparable Tables
# (only Tables logged to the same key can be compared)
VAL_TABLE_NAME = "predictions" 

# hyperparams set low for demo/training speed
# if you set these higher, be mindful of how many items are in
# the dataset artifacts you chose by setting the SIZE at the top
NUM_TRAIN = BALANCED_SPLITS["train"]*4
NUM_VAL = BALANCED_SPLITS["val"]*4

# enforced max for this is ceil(NUM_VAL/batch_size)
NUM_LOG_BATCHES = 16

# ARTIFACTS CONFIG
#------------------------
# training data artifact to load
TRAIN_DATA_AT = PREFIX + "_80-10-10_" + str(TOTAL_IMAGES)

# model name
# if you want to train a sufficiently different model, give this a new name
# to start a new lineage for the model, instead of just incrementing the
# version of the old model
MODEL_NAME = "iv3_finetuned"

# folder in which to save the final, trained model
# if you want to train a sufficiently different model, give this a new name
# to start a new lineage for the model, instead of just incrementing the
# version of the old model
SAVE_MODEL_DIR = "finetune_iv3_keras"

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from wandb.keras import WandbCallback

# experiment configuration saved to W&B
CFG = {
  "num_train" : NUM_TRAIN,
  "num_val" : NUM_VAL,
  "num_classes" : 4,
  "fc_size" : 1024,
  "epochs" : NUM_EPOCHS,
  "batch_size" : 32,

  # inceptionV3 settings
  "img_width" : 299,
  "img_height": 299
}

# number of validation data batches to log/use when computing metrics
# at the end of each epoch
max_log_batches = int(np.ceil(float(CFG["num_val"])/float(CFG["batch_size"])))
# change this min to max to log ALL the available images to a Table
CFG["num_log_batches"] = min(max_log_batches, NUM_LOG_BATCHES)

def finetune_inception_model(fc_size, num_classes):
  """Load InceptionV3 with ImageNet weights, freeze it,
  and attach a finetuning top for this classification task"""
  # load InceptionV3 as base
  base = InceptionV3(weights="imagenet", include_top="False")
  # freeze base layers
  for layer in base.layers:
    layer.trainable = False
  x = base.get_layer('mixed10').output 

  # attach a fine-tuning layer
  x = GlobalAveragePooling2D()(x)
  x = Dense(fc_size, activation='relu')(x)
  guesses = Dense(num_classes, activation='softmax')(x)

  model = Model(inputs=base.input, outputs=guesses)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def train():
  """ Main training loop which freezes the InceptionV3 layers of the model
  and only trains the new top layers on the new data. A subsequent training
  phase might unfreeze all the layers and finetune the whole model on the new data""" 
  run = wandb.init(project=PROJECT_NAME, name=RUN_NAME, job_type="train", config=CFG)
  cfg = wandb.config

  # locate and download training and validation data
  data_at = TRAIN_DATA_AT + ":latest"
  data = run.use_artifact(data_at, type="balanced_data")
  data_dir = data.download()
  train_dir = os.path.join(data_dir, "train")
  val_dir = os.path.join(data_dir, "val")

  # create train and validation data generators
  train_datagen = ImageDataGenerator(
      rescale=1. / 255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)
  val_datagen = ImageDataGenerator(rescale=1. / 255)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(cfg.img_width, cfg.img_height),
    batch_size=cfg.batch_size,
    class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(cfg.img_width, cfg.img_height),
    batch_size=cfg.batch_size,
    class_mode='categorical',
    shuffle=False)

  # instantiate model and callbacks
  model = finetune_inception_model(cfg.fc_size, cfg.num_classes)
  callbacks = [WandbCallback(), ValLog(val_generator, cfg.num_log_batches)]

  # train!
  model.fit(
    train_generator,
    steps_per_epoch = cfg.num_train // cfg.batch_size,
    epochs=cfg.epochs,
    validation_data=val_generator,
    callbacks = callbacks,
    validation_steps = cfg.num_val // cfg.batch_size)

  # save trained model as artifact
  trained_model_artifact = wandb.Artifact(
            MODEL_NAME, type="model",
            description="finetuned inception v3",
            metadata=dict(cfg))
  
  model.save(SAVE_MODEL_DIR)
  trained_model_artifact.add_dir(SAVE_MODEL_DIR)
  run.log_artifact(trained_model_artifact)
  run.finish()

class ValLog(Callback):
  """ Custom callback to log validation images
  at the end of each training epoch"""
  def __init__(self, generator=None, num_log_batches=1):
    self.generator = generator
    self.num_batches = num_log_batches
    # store full names of classes
    self.flat_class_names = [k for k, v in generator.class_indices.items()]

  def on_epoch_end(self, epoch, logs={}):
    # collect validation data and ground truth labels from generator
    val_data, val_labels = zip(*(self.generator[i] for i in range(self.num_batches)))
    val_data, val_labels = np.vstack(val_data), np.vstack(val_labels)

    # use the trained model to generate predictions for the given number
    # of validation data batches (num_batches)
    val_preds = self.model.predict(val_data)
    true_ids = val_labels.argmax(axis=1)
    max_preds = val_preds.argmax(axis=1)

    # log validation predictions alongside the run
    columns=["id", "image", "guess", "truth"]
    for a in self.flat_class_names:
      columns.append("score_" + a)
    predictions_table = wandb.Table(columns = columns)
    
    # log image, predicted and actual labels, and all scores
    for filepath, img, top_guess, scores, truth in zip(self.generator.filenames,
                                                       val_data, 
                                                       max_preds, 
                                                       val_preds,
                                                       true_ids):
      img_id = filepath.split('/')[-1].split(".")[0]
      row = [img_id, wandb.Image(img), 
             self.flat_class_names[top_guess], self.flat_class_names[truth]]
      for s in scores.tolist():
        row.append(np.round(s, 4))
      predictions_table.add_data(*row)
    wandb.run.log({VAL_TABLE_NAME : predictions_table})

In [15]:
train()

wandb: Downloading large artifact cornset_80-10-10_2000:latest, 71.13MB. 2001 files... Done. 0:0:0.9


Found 1600 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
96124928/96112376 [==============================] - 1s 0us/step


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/5
50/50 [==============================] - 61s 933ms/step - loss: 0.7411 - accuracy: 0.7500 - val_loss: 0.3990 - val_accuracy: 0.7865 - _timestamp: 1658974457.0000 - _runtime: 63.0000
Epoch 2/5
50/50 [==============================] - 42s 836ms/step - loss: 0.2828 - accuracy: 0.8863 - val_loss: 0.4039 - val_accuracy: 0.8021 - _timestamp: 1658974503.0000 - _runtime: 109.0000
Epoch 3/5
50/50 [==============================] - 43s 868ms/step - loss: 0.2426 - accuracy: 0.9006 - val_loss: 0.3445 - val_accuracy: 0.8385 - _timestamp: 1658974544.0000 - _runtime: 150.0000
Epoch 4/5
50/50 [==============================] - 41s 818ms/step - loss: 0.2116 - accuracy: 0.9137 - val_loss: 0.3604 - val_accuracy: 0.8385 - _timestamp: 1658974587.0000 - _runtime: 193.0000
Epoch 5/5
50/50 [==============================] - 42s 836ms/step - loss: 0.2025 - accuracy: 0.9181 - val_loss: 0.4131 - val_accuracy: 0.822

wandb: Adding directory to artifact (./finetune_iv3_keras)... Done. 1.0s


accuracy,▁▇▇██
epoch,▁▃▅▆█
loss,█▂▂▁▁
val_accuracy,▁▃██▆
val_loss,▇▇▁▃█
GFLOPS,5.72554
accuracy,0.91812
best_epoch,2
best_val_loss,0.34445
epoch,4
loss,0.20249


## Test Model (Run Inference)




In [18]:
# optional globals to modify
# set to a custom name to help keep your experiments organized
RUN_NAME = "" 
# change this if you'd like start a new set of comparable Tables
# (only Tables logged to the same key can be compared)
TEST_TABLE_NAME = "test_results" 

from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
import os

MODEL_NAME = "iv3_finetuned"
# location of test data from our original split
# should match SPLIT_DATA_AT
TEST_DATA_AT = "_".join([PREFIX, "80-10-10", str(TOTAL_IMAGES)])

run = wandb.init(project=PROJECT_NAME, job_type="inference", name=RUN_NAME)
model_at = run.use_artifact(MODEL_NAME + ":latest")
model_dir = model_at.download()
print("model: ", model_dir)
model = keras.models.load_model(model_dir)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# download latest version of test data
test_data_at = run.use_artifact(TEST_DATA_AT + ":latest")
test_dir = test_data_at.download()
test_dir += "/test/"

#class_names = ["Gray Leaf Spot", "Blight", "Common Rust", "Healthy"]
#class_names = ["Common Rust", "Gray Leaf Spot", "Blight", "Healthy"]
class_names = ["Blight", "Common Rust","Gray Leaf Spot", "Healthy"]

# load test images
imgs = []
filenames = []
class_labels = os.listdir(test_dir)
truth = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join(test_dir, l))
  for img in imgs_per_class:
    # track the image id
    filenames.append(img.split(".")[0])
    truth.append(l)
    img_path = os.path.join(test_dir, l, img)
    img = image.load_img(img_path, target_size=(299, 299))
    img = image.img_to_array(img)
    # don't forget to rescale test images to match the range of inputs
    # to the network
    img = np.expand_dims(img/255.0, axis=0)
    imgs.append(img)

# predict on test data and bin predictions by guessed label 
preds = {}
imgs = np.vstack(imgs)
classes = model.predict(imgs, batch_size=32)
for c in classes:
  class_id = np.argmax(c)
  if class_id in preds:
    preds[class_id] += 1
  else:
    preds[class_id] = 1

# log inference results as a Table to the run workspace
columns=["id", "image", "guess", "truth"]
for a in class_names:
  columns.append("score_" + a)
test_dt = wandb.Table(columns = columns)

# store all the scores for each image
for img_id, i, t, c in zip(filenames, imgs, truth, classes):
  guess = class_names[np.argmax(c)]
  row = [img_id, wandb.Image(i), guess, t]
  for c_i in c.tolist():
    row.append(np.round(c_i, 4))
  test_dt.add_data(*row)
  
run.log({TEST_TABLE_NAME : test_dt})
print("Quick distribution of predicted classes: ")
print(preds)
run.finish()

wandb: Downloading large artifact iv3_finetuned:latest, 112.33MB. 4 files... Done. 0:0:0.1


model:  ./artifacts/iv3_finetuned:v18


wandb: Downloading large artifact cornset_80-10-10_2000:latest, 71.13MB. 2001 files... Done. 0:0:0.2


Quick distribution of predicted classes: 
{2: 38, 0: 57, 1: 57, 3: 48}


##Hyperparametric Tuning (Sweep)

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize',   
    },
    'parameters': {
        'batch_size': {
            'values': [32,64]
        },
        'epoch': {
            'values': [10,15,20]
        }, 
        'criterion': {
            'values': ["BinaryCrossentropy", "MeanSquaredError"]
        },
         'optmizer': {
            'values': ["sgd", "adam", "rmsprop", "adagrad", "MeanSquaredError"]
        },
    }
}

In [ ]:
# Initialize a new sweep
# Arguments:
#     – sweep_config: the sweep config dictionary defined above
#     – entity: Set the username for the sweep
#     – project: Set the project name for the sweep
sweep_id = wandb.sweep(sweep_config, entity="gabrieljordao", project="corn_leaf_photos")

Create sweep with ID: hkwtdwts
Sweep URL: https://wandb.ai/gabrieljordao/corn_leaf_photos/sweeps/hkwtdwts


In [ ]:
# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
wandb.agent(sweep_id = sweep_id, function=train, count=10)

wandb: Agent Starting Run: x995j10t with config:
wandb: 	batch_size: 32
wandb: 	criterion: MeanSquaredError
wandb: 	epoch: 15
wandb: 	optmizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Downloading large artifact cornset_80-10-10_2000:latest, 91.09MB. 2001 files... Done. 0:0:1.1


Found 1600 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
96124928/96112376 [==============================] - 1s 0us/step


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/5
35/50 [====================>.........] - ETA: 1:33 - loss: 0.8772 - accuracy: 0.6973